In [1]:
##STEP 1: Open all files

import pandas as pd

anime_train = pd.read_csv(".//Database//Anime_train.csv")
anime_test = pd.read_csv(".//Database//Anime_test.csv")

char_train = pd.read_csv(".//Database//Char_train.csv")
char_test = pd.read_csv(".//Database//Char_test.csv")

impath_train = ".//Database//char train//"
impath_test = ".//Database//char test//"


#Original format for characters:
anime_train["Main Characters"] = anime_train["Main Characters"].apply(lambda x:[int(y) for y in x[1:-1].split(", ")])
anime_test["Main Characters"] = anime_test["Main Characters"].apply(lambda x:[int(y) for y in x[1:-1].split(", ")])

In [2]:
###ALL GENERAL VALUES:

batch_size = 16
max_length_syn = 128
max_length_char = 256
epochs = 30 #1000

learning_rate = 5e-5
eps_value = 1e-8
syn_model_name_or_path = 'gpt2'
char_model_name_or_path = 'gpt2'
img_model_name_or_path = 'microsoft/resnet-50'

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"torch running in {device}")

from transformers import set_seed
set_seed(42)

torch running in cpu


In [3]:
###STEP 2: Open all datasets

from torch.utils.data import Dataset
import numpy as np
import os
from PIL import Image 
import numpy as np

from tqdm import tqdm

def scaler(values, ratio): #Only works with positive values.
    return [(v - min(values)) * (ratio / (max(values) - min(values))) for v in values]

class AnimeDataset(Dataset):
    def __init__(self, df, char_df, mchars, img_path, transform=None):

        self.labels = []
        for x in df.index:
            self.labels.append(df["Score"][x])
        
        self.syn = []
        for x in df.index:
            self.syn.append(df["Synopsis"][x].split("(source")[0].split("(Source")[0].split("[Written")[0])

        self.mchars = []
        for x in df.index:
            list_chars = mchars[x]
            chars = []
            for y in list_chars:
                personaje = char_df[char_df["MAL ID"] == y].reset_index()
                chars.append(personaje["Description"][0].split("(source")[0].split("(Source")[0].split("[Written")[0])
            self.mchars.append(' '.join(chars))

        self.transform = transform
        self.img = []
        for x in tqdm(df.index, desc="Concatenating portraits"):
            list_chars = [str(y)+".png" for y in mchars[x]]
            chars = []
            for y in list_chars:
                img_name = img_path+y
                personaje = Image.open(img_name).convert('RGB') #Abrir la imagen
                chars.append(personaje)

            #Appending all portraits, horizontally.
            widths, heights = zip(*(img.size for img in chars))
            retratos = Image.new('RGB', (sum(widths), max(heights)))
            
            x_offset = 0
            for img in chars:
                retratos.paste(img, (x_offset, 0))
                x_offset += img.width

            if self.transform:
                retratos = self.transform(retratos)

            self.img.append(retratos)

        self.labels = scaler(self.labels, 1)

        return
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        return {'synopsis':self.syn[item], 'mchar':self.mchars[item], 'img':self.img[item], 'label':self.labels[item]}
    

transf = None
####It can be:
##
##transf = transforms.Compose([
##    transforms.Resize(size),        # You should first define a tuple "size". This can aid in memory saving.
##    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images. Values are just illustrative.
##    ])

train_dataset = AnimeDataset(df=anime_train, char_df=char_train, mchars=anime_train["Main Characters"], img_path=impath_train, transform=transf)
test_dataset = AnimeDataset(df=anime_test, char_df=char_test, mchars=anime_test["Main Characters"], img_path=impath_test, transform=transf)

Concatenating portraits: 100%|██████████| 1439/1439 [00:09<00:00, 147.72it/s]


In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

class tfidf():
    def __init__(self):
        self.vec = TfidfVectorizer()

    def prepros(self, texto):
        return re.sub(r'[^A-Za-z0-9\s]', ' ', texto)
    
    def vectorizer(self, texts):
        pad = 750
        prepros_docs = [self.prepros(x) for x in texts]
        x = self.vec.fit_transform(prepros_docs).toarray()

        padding = torch.zeros(x.shape[0], pad)
        x_tensor = torch.cat((torch.Tensor(x), padding), dim=1)[:,:pad]
        return x_tensor
    

import torchvision.transforms as transforms

class img_tensorification():
    def __init__(self):
        self.transform = transforms.Compose([transforms.PILToTensor()])
    
    def vectorizer(self, imgs):
        pad = 750
        padding = torch.zeros(1, pad)
        img_tensors = [torch.cat((self.transform(im).reshape(1, -1), padding), dim=1)[:, :pad] for im in imgs]
        img_tensors = torch.cat(img_tensors, dim=0)
        return img_tensors

In [5]:
###STEP 3: Collate all the data into a unified input

class AnimeRegressionCollator(object):
    def __init__(self, img_vectorizer, syn_vectorizer, char_vectorizer):
        self.syn_vectorizer = syn_vectorizer
        self.char_vectorizer = char_vectorizer
        self.img_vectorizer = img_vectorizer
        return

    def __call__(self, sequences):
        labels = [sequence['label'] for sequence in sequences]

        synopsis = [sequence['synopsis'] for sequence in sequences]
        char = [sequence['mchar'] for sequence in sequences]
        img = [sequence['img'] for sequence in sequences]

        syn_v = self.syn_vectorizer
        char_v = self.char_vectorizer
        img_v = self.img_vectorizer

        inputs = {'portraits': img_v.vectorizer(img)}
        inputs.update({'synopsis': syn_v.vectorizer(synopsis)})
        inputs.update({'descriptions': char_v.vectorizer(char)})
        inputs.update({'labels': torch.tensor(np.array(labels), dtype=torch.float)})
        return inputs

In [6]:
img_v = img_tensorification()
syn_v = tfidf()
char_v = tfidf()

regression_collator = AnimeRegressionCollator(img_vectorizer= img_tensorification(), 
                                              syn_vectorizer= tfidf(), 
                                              char_vectorizer= tfidf())

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=regression_collator)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=regression_collator)

In [7]:
###STEP 4: Initialize the neural network

import torch.nn as nn
from transformers import GPT2Model, ResNetModel

class TradNet(nn.Module):
    def __init__(self, pad):
        super(TradNet, self).__init__()
        self.pad = pad

        #CHARACTERS:
        self.linear_regression = nn.Sequential(
                nn.Linear(self.pad,1000, bias=True),
                nn.Tanh(),
                nn.Linear(1000,500, bias=True),
                nn.Tanh(),
                nn.Linear(500,250, bias=True),
                nn.Tanh(),
                nn.Linear(250,100, bias=True),
                nn.Tanh(),
                nn.Linear(100,1, bias=True),
                nn.Softmax(dim=0)
        )
        
    def forward(self, syn_input_ids, char_input_ids, img_input_ids):

        #print(syn_input_ids.shape, char_input_ids.shape, img_input_ids.shape)

        all = torch.concat((syn_input_ids, char_input_ids, img_input_ids), dim=1)
        if all.shape[1] >= self.pad:
            all = all[:, :self.pad]
        else:
            padding = torch.zeros(1, self.pad)
            all = torch.cat((all, padding), dim=1)[:, :self.pad]

        logits = self.linear_regression(all)

        return logits
    

model = TradNet(pad=2250)
model.to(device)

TradNet(
  (linear_regression): Sequential(
    (0): Linear(in_features=2250, out_features=1000, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1000, out_features=500, bias=True)
    (3): Tanh()
    (4): Linear(in_features=500, out_features=250, bias=True)
    (5): Tanh()
    (6): Linear(in_features=250, out_features=100, bias=True)
    (7): Tanh()
    (8): Linear(in_features=100, out_features=1, bias=True)
    (9): Softmax(dim=0)
  )
)

In [8]:
def training_loop(train_loader, predictions, true_labels, optimizer_, scheduler_, device_, loss_fn):
    global model

    model.train()

    total_loss = 0

    for batch in tqdm(train_loader, total=len(train_loader), desc="Batch"):

        true_labels += batch['labels'].numpy().flatten().tolist()

        model.zero_grad()
        
        ##INPUTS:
        
        syn_input_ids = batch['synopsis'].type(torch.long).to(device_)
        char_input_ids = batch['descriptions'].type(torch.long).to(device_)

        img_input_ids = batch['portraits'].type(torch.float).to(device_)

        
        logits = model(syn_input_ids=syn_input_ids, char_input_ids=char_input_ids, img_input_ids=img_input_ids).to(device_)

        predictions_loss = logits.squeeze()

        lbels = torch.Tensor(batch['labels'].float()).to(device_)
        loss = loss_fn(predictions_loss, lbels)
        total_loss += loss.item()

        #optimizer_.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer_.step()
        scheduler_.step()

        predictions += predictions_loss

    avg_epoch_loss = total_loss / len(train_loader)
    return true_labels, predictions, avg_epoch_loss


def validation(test_loader, device_, loss_fn):
    global model

    predictions = []
    true_labels = []

    total_loss = 0

    model.eval()

    for batch in tqdm(test_loader, total=len(test_loader)):
        true_labels += batch['labels'].numpy().flatten().tolist()
        
        ##INPUTS:
        syn_input_ids = batch['synopsis'].type(torch.long).to(device_)
        char_input_ids = batch['descriptions'].type(torch.long).to(device_)

        img_input_ids = batch['portraits'].type(torch.float).to(device_)

        with torch.no_grad(): # Telling the model not to compute or store gradients, saving memory and speeding up validation
            outputs = model(syn_input_ids=syn_input_ids, char_input_ids=char_input_ids, img_input_ids=img_input_ids).to(device_)
            logits = outputs

            predictions += logits.squeeze().detach().cpu().tolist()
            predictions_loss = torch.Tensor(logits.squeeze().detach().cpu()).to(device_)

            loss = loss_fn(predictions_loss, torch.Tensor(batch['labels'].float()).to(device_))

            total_loss += loss.item()

    avg_epoch_loss = total_loss / len(test_loader)

    return true_labels, predictions, avg_epoch_loss

In [9]:
from Ca_Naxca import regression_report
from transformers import get_linear_schedule_with_warmup

##TRAIN THE MODEL.

optimizer_ = torch.optim.AdamW(model.parameters(), lr = learning_rate, eps = eps_value)
total_steps = len(train_dataloader) * epochs
scheduler_ = get_linear_schedule_with_warmup(optimizer_, num_warmup_steps = 0, num_training_steps = total_steps)
loss_fn = nn.MSELoss()  # Loss function for regression problems


trainval = []

for epoch in tqdm(range(epochs), desc="Epoch"):
    true_labels = []
    predictions = []
    avg_epoch_loss = 0

    # Train the model:
    true_labels, predictions, train_loss = training_loop(train_dataloader, predictions, true_labels, optimizer_, scheduler_, device, loss_fn)
    # Test the model:
    valid_labels, valid_predict, val_loss = validation(test_dataloader, device, loss_fn)

    # Is it good enough?
    reporte = regression_report(valid_labels, valid_predict, [i for i in range(len(valid_labels))])
    reps = reporte.display()
    print(reps)
    reps.to_csv(".//final_reports//trad//reporte_full.csv")

    for x in [[valid_labels[i], valid_predict[i]] for i in range(10)]:
        print(x)

    print("  train_loss: %.5f - val_loss: %.5f "%(train_loss, val_loss))
    print()
    trainval.append([train_loss, val_loss])


import json

jsonfile = open(".//final_reports//trad//train_val_loss.json", "w")
json.dump(trainval, jsonfile)
jsonfile.close()

Epoch:   3%|▎         | 1/30 [00:22<10:47, 22.33s/it]

   Mean Absolute Error  Mean Squared Error   R2 score  \
0             0.632716            0.413067 -30.811169   

                                        Spearman  \
0  (0.10953892131977355, 3.1245802573644394e-05)   

                                         Pearson  \
0  (0.11281014144664404, 1.7920736736786908e-05)   

                                    Kendall-Tau  
0  (0.07306393150639916, 3.401097945852815e-05)  
[0.7902777791023254, 0.07204839587211609]
[0.9750000238418579, 0.0676451101899147]
[0.6305555701255798, 0.05363726615905762]
[0.7666666507720947, 0.06375724077224731]
[0.8402777910232544, 0.07845451682806015]
[0.875, 0.06480666995048523]
[0.6111111044883728, 0.06299734860658646]
[0.7222222089767456, 0.053323499858379364]
[0.6458333134651184, 0.05217423290014267]
[0.8930555582046509, 0.09251859039068222]
  train_loss: 0.39932 - val_loss: 0.41302 



Epoch:   7%|▋         | 2/30 [00:43<10:10, 21.82s/it]

   Mean Absolute Error  Mean Squared Error  R2 score  \
0             0.632716            0.412959 -30.80288   

                                        Spearman  \
0  (0.13626713423070466, 2.1153158304822687e-07)   

                                         Pearson  \
0  (0.14950075526471462, 1.2100214860078543e-08)   

                                    Kendall-Tau  
0  (0.09056878799171683, 2.779357781985981e-07)  
[0.7902777791023254, 0.05760006606578827]
[0.9750000238418579, 0.07755430787801743]
[0.6305555701255798, 0.05204545333981514]
[0.7666666507720947, 0.055082473903894424]
[0.8402777910232544, 0.07455141097307205]
[0.875, 0.06878554821014404]
[0.6111111044883728, 0.05536554753780365]
[0.7222222089767456, 0.04436374083161354]
[0.6458333134651184, 0.047919683158397675]
[0.8930555582046509, 0.10185369849205017]
  train_loss: 0.39928 - val_loss: 0.41291 



Epoch:  10%|█         | 3/30 [01:07<10:09, 22.57s/it]

...

   Mean Absolute Error  Mean Squared Error   R2 score  \
0               0.6327            0.412892 -30.797686   

                                      Spearman  \
0  (0.1896129572756978, 4.093168492376794e-13)   

                                        Pearson  \
0  (0.17564083868320315, 1.956710516962146e-11)   

                                    Kendall-Tau  
0  (0.12650481218664744, 7.192071297452033e-13)  
[0.7902777791023254, 0.07132725417613983]
[0.9750000238418579, 0.1041082963347435]
[0.6305555701255798, 0.06918632239103317]
[0.7666666507720947, 0.05749712139368057]
[0.8402777910232544, 0.06287378072738647]
[0.875, 0.06882830709218979]
[0.6111111044883728, 0.05535203963518143]
[0.7222222089767456, 0.04778503254055977]
[0.6458333134651184, 0.043311867862939835]
[0.8930555582046509, 0.05968809872865677]
  train_loss: 0.39814 - val_loss: 0.41284 



Epoch:  97%|█████████▋| 29/30 [12:59<00:26, 26.30s/it]

   Mean Absolute Error  Mean Squared Error   R2 score  \
0             0.632691            0.412828 -30.792809   

                                       Spearman  \
0  (0.19640418708824642, 5.592613181941376e-14)   

                                        Pearson  \
0  (0.18534330665701873, 1.378315886603478e-12)   

                                   Kendall-Tau  
0  (0.130855472981259, 1.1475862046848718e-13)  
[0.7902777791023254, 0.07721468061208725]
[0.9750000238418579, 0.11291643232107162]
[0.6305555701255798, 0.0699954628944397]
[0.7666666507720947, 0.05901720002293587]
[0.8402777910232544, 0.0642930269241333]
[0.875, 0.061919428408145905]
[0.6111111044883728, 0.05556878447532654]
[0.7222222089767456, 0.049317896366119385]
[0.6458333134651184, 0.04329591616988182]
[0.8930555582046509, 0.06159442663192749]
  train_loss: 0.39805 - val_loss: 0.41278 



Epoch: 100%|██████████| 30/30 [13:24<00:00, 26.83s/it]

   Mean Absolute Error  Mean Squared Error   R2 score  \
0              0.63269            0.412863 -30.795464   

                                       Spearman  \
0  (0.19483599679434838, 8.913466374155305e-14)   

                                         Pearson  \
0  (0.18316876463253481, 2.5300129696611337e-12)   

                                    Kendall-Tau  
0  (0.1297859724206788, 1.8139730874613574e-13)  
[0.7902777791023254, 0.07491353154182434]
[0.9750000238418579, 0.1133069097995758]
[0.6305555701255798, 0.06819849461317062]
[0.7666666507720947, 0.059334687888622284]
[0.8402777910232544, 0.06433317810297012]
[0.875, 0.0626467913389206]
[0.6111111044883728, 0.05606323480606079]
[0.7222222089767456, 0.051430247724056244]
[0.6458333134651184, 0.04208425059914589]
[0.8930555582046509, 0.0588628388941288]
  train_loss: 0.39792 - val_loss: 0.41282 

